## 0. Prompts

In [ ]:
aim_research = """

                The aim of this study is to investigate how university students use, value, and trust feedback generated by Generative Artificial Intelligence (GenAI) 
                compared to feedback from teachers. Specifically, the research seeks to understand the different roles that GenAI and teacher feedback play in 
                higher education, the factors that influence students’ perceptions of their usefulness, reliability, and trustworthiness, and the ways in which these 
                forms of feedback shape students’ learning experiences, emotions, and engagement with feedback processes.  

                """


actor_codes =  """
                # Code: A  

                **Explanation:** GenAI (also variants such as AI, ChatGPT, etc.)  
                **Representative quotes:**  
                - “GenAI feedback is...” (AO2068)  
                - “Sometimes the feedback can be incorrect from AI..” (AO2068)  

                ---

                # Code: T  

                **Explanation:** Teacher (also variants such as lecturer, tutor, staff, etc.)  
                **Representative quote:**  
                - “The teacher will usually give very accurate feedback…” (AO2068)  

                ---

                # Code: B  

                **Explanation:** Both (clear reference to both actors)  
                **Representative quotes:**  
                - “Both forms of feedback were similar.” (AA2068)  
                - “Sometimes the feedback can feel vague with both teachers and GenAI which creates confusion.” (AO2523)  

              """
               
comparator_codes = """

                    # Code: More  

                    **Explanation:** When the comment indicated that the actor was associated with more (volume, frequency, impact) of the characteristic.  
                    **Representative quotes:**  
                    - “The speed in which AI can give responses is much faster” (G29)  
                    - “Teacher feedback tended to be more deep and considering the context”  

                    ---

                    # Code: Less  

                    **Explanation:** As above but in reference to less or smaller volume, frequency or impact of the characteristic.  
                    **Representative quotes:**  
                    - “I would not trust GenAI to give me the correct answer or feedback” (AW17)  
                    - “Teacher: - Limited time, so minimal feedback - Less detailed - Less emotionally supporting” (AO2071)  

                    ---

                    # Code: Similar  

                    **Explanation:** When the comment made an explicit statement that both GenAI and Teachers were similar in some way (e.g., they were similarly useful).  
                    **Representative quotes:**  
                    - “Both forms of feedback were similar.” (AA2068)  
                    - “Sometimes the feedback can feel vague with both teachers and GenAI which creates confusion.” (AO2523)  


                    """
                    
characteristic_codes = """
                        # Theme: Processes  

                        ## Sub-theme: Access  

                        ### Code: Ease  
                        **Definition:** Is easily accessible, available, or convenient.  
                        **Representative quote:**  
                        - “It’s easier than trying to reach professors and teachers” (G22)  

                        ### Code: Speed  
                        **Definition:** Is immediate, instant or fast. Described as time-efficient, saving time.  
                        **Representative quotes:**  
                        - “the speed in which AI can give responses is much faster” (G29)  
                        - “It helped save time spent on the ‘less important’ aspects of assignments” (G829)  

                        ### Code: Volume  
                        **Definition:** Reference to quantity, volume, scale of information or feedback interactions (e.g., could ask many questions and get lots of answers).  
                        **Representative quotes:**  
                        - “much more accessible meaning I could ask a lot more questions” (G37)  
                        - “Teacher gives very little feedback” (G91)  

                        ## Sub-theme: Timing  

                        ### Code: Before submission  
                        **Definition:** Provides information during the production of an assignment task.  
                        **Representative quote:**  
                        - “GenAI feedback is instant and available prior to assessment submissions, which helps improve writing as it provides feedback on drafting for final submission.” (G64)  

                        ### Code: After submission  
                        **Definition:** Provides information after the submission of an assignment task.  
                        **Representative quotes:**  
                        - “Teacher feedback is helpful in hindsight” (U64)  
                        - “Teacher feedback however is provided after assessment is submitted which is helpful in how the writing COULD have been improved, but is not applicable help since the assessment is already completed.” (G64)  

                        ## Sub-theme: Effort  

                        ### Code: Less effort  
                        **Definition:** Reduces effort associated with the task. This may be making a task easier in some way, or it may refer to doing the task entirely.  
                        **Representative quotes:**  
                        - “GenAI can save lots of effort in searching and writing” (G3453)  
                        - “It makes my work easier” (G1379)  
                        - “Some easy problems can be solved by AI” (G2101)  
                        - “GenAI helps me get started with writing as I usually get a mind block – can’t start off a paragraph” (G2220)  

                        ---

                        # Theme: Sense-making  

                        ## Sub-theme: Sense-making 

                        ### Code: Understanding  
                        **Definition:** Helps me understand.  
                        **Representative quote:**  
                        - “It has helped me organise my thoughts and cohesively express them and understand concepts/events at a basic level” (G68)  

                        ### Code: Reflection  
                        **Definition:** Helps me reflect.  
                        **Representative quote:**  
                        - “AI made me more reflective” (U1289)  

                        ### Code: Progress    
                        **Definition:** Helps me know how I’m going.  
                        **Representative quote:**  
                        - “It helped me understand how I was fairing” (U91)  

                        ---

                        # Theme: Information  

                        ## Sub-theme: Quality  

                        ### Code: Specificity  
                        **Definition:** Provides specific or detailed information. Antonyms: vague, generic.  
                        **Representative quotes:**  
                        - “GenAI isn't specific or …” (G50)  
                        - “GenAI offered specific answers unlike my vague teachers” (AO302)  

                        ### Code: In-depth  
                        **Definition:** Provides in-depth or nuanced information. Antonyms: superficial, broad.  
                        **Representative quotes:**  
                        - “GENAI does not have the nuance, it is broad” (G11)  
                        - “Response time and depth of feedback from GenAI was much better but also less reliable” (AO4363)  
                        - “GenAI is very fast and can provide a wealth of information and resources, but it lacks in-depth understanding and emotional support. Teacher feedback is more specific and thorough, offering more background knowledge and experience” (AO4554)  

                        ### Code: Understandable  
                        **Definition:** Information is presented in an understandable, digestible, and comprehensible way.  
                        **Representative quotes:**  
                        - “GenAI is great for summarizing content and dividing large amounts of material into digestible bullet points.” (G20)  
                        - “I like to use GenAI to give me multiple walkthroughs/analogies for my learning alongside what my teachers provide. I have found it very helpful and in general can provide more ways of understanding the content than just what my teachers provide.” (G2311)  

                        ### Code: Relevance  
                        **Definition:** Information is relevant, on topic.  
                        **Representative quote:**  
                        - “AI always gives irrelevant and incomplete answers” (G95)  

                        ### Code: Contextualised  
                        **Definition:** Provides information that is adapted for the context of an assignment, rubrics, discipline, or class. (Not relational).  
                        **Representative quotes:**  
                        - “Teacher feedback tended to be more deep and considering the context”  
                        - “Teacher feedback is specific to their marking requirements”  
                        - “If you want to know more, teacher will tell you what is and isn’t relevant to what you need to know for your field of study”  

                        ### Code: Utility  
                        **Definition:** Provides useful, usable, or helpful information.  
                        **Representative quotes:**  
                        - “GenAI can directly tell me the implementation process or answer”  
                        - “GenAI offers immediate access to information, explanations, and creative ideas, which can be helpful for quick problem-solving and exploring new concepts.”  

                        ### Code: Reliable  
                        **Definition:** Provides information that is accurate, precise, reliable, or trustworthy.  
                        **Representative quote:**  
                        - “GenAI can be unreliable when providing feedback for scientific writings” (AW36)  

                        ### Code: Objective  
                        **Definition:** Is objective in its feedback, judgment, or evaluation. Antonyms: subjective, biased.  
                        **Representative quotes:**  
                        - “There may be subjective bias in teacher given feedback whereas AI is more objective and factual.” (G7089)  
                        - “Teacher’s feedback is always… dependent on the marker and their opinions. GenAI is objective, through its understanding of the internet.” (G2271)  
                        - “Since GenAI has no bias or preference, it purely gives feedback on what it reads.” (G1358)  
                        - “The teacher could be inconsistent and interpret the criteria in their own way due to misreading and being poorly informed maybe due to lack of time.” (G1611)  

                        ---

                        ## Sub-theme: Tone  

                        ### Code: Positivity  
                        **Definition:** Makes positive statements, people pleaser
                        **Representative quotes:**  
                        - “It is also a people pleaser” (G11)  
                        - “AI can be as nice as you ask it to be” (AA1729)  

                        ### Code: Negativity  
                        **Definition:** Makes edits or provides statements that are perceived as negative in tone (e.g. dismissive, insulting, uncaring).  
                        **Representative quotes:**  
                        - “GenAI feedback was always polite and constructive, but unfortunately I can’t say the same about all teacher/staff feedback.” (G39)  
                        - “Whereas my teacher will have edits to my work.” (G23)  

                        ---

                        # Theme: Feeling  
                        
                        ## Sub-theme: Feeling  

                        ### Code: Positive  
                        **Definition:** Makes me feel a positive feeling. Synonyms: Encouraging, motivating.  
                        **Representative quotes:**  
                        - “AI made me feel good” (G23)  
                        - “The AI feels like it genuinely cares and wants to help but the teachers do not” (AA1791)  
                        - “I feel more important and loved by AI than my teachers” (AA1792)  

                        ### Code: Negative  
                        **Definition:** Makes me feel a negative feeling. Synonyms: Frustrating. 
                        **Representative quotes:**  
                        - “Teachers make me feel bad and dumb” (G23)  
                        - “Feedback with harsh, unhelpful wording from teachers/staff… had a greater impact on me emotionally.” (G39)  

                        ### Code: No impact
                        **Definition:** Makes me feel nothing or indifferent. Has no impact on my emotions or feelings.
                        **Representative quotes:**  
                        - “GenAI makes me feel nothing. I know it is AI working off machine learning based on what it has learnt before.” (AA11)  
                        - “GenAI has no impact on my feelings as it’s not a person” (AA1442)  
                        - “ChatGPT is emotionless so it could say it was awful and I wouldn’t care, but I’d prefer not to hear that from a teacher” (AA1999)  

                        ---

                        # Theme: Relational  

                        ## Sub-theme: Relational
                         
                        ### Code: Personal   
                        **Definition:** Is supportive. Concerns relational type statements (e.g. the “teacher knows me”’). Closer/ more-distant or embodied (e.g. “I like being face to face with my teacher”). Some statements are about AI being more in tune, more personalised, personal to tastes in language, communication and "learning styles".
                        **Representative quotes:**  
                        - “GenAI can’t have a person-to-person conversation with me” (AW10)  
                        - “GenAI: Offers highly personalized learning experiences, adapting to your specific questions, pace, and learning style” (G6678)  
                        - “Teacher… knows where you are up to in your learning” (AO4329)  
                        - “A teacher provides personalized guidance, context, and the ability to adapt explanations based on individual needs, fostering a deeper understanding and critical thinking.” (G2020)  

                        ### Code: Risky  
                        **Definition:** Makes me vulnerable to their judgement, causes me shame. Possibly related to power, position, self. Synonyms: Implies personal consequence, at risk. Antonyms: non-judgemental, no shame, preserves self, prevents vulnerability
                        **Representative quotes:**  
                        - “I have severe anxiety and issues with asking and receiving help… having AI/LLMs provide text responses help” (G75)  
                        - “AI provides a safe space to ask any and all questions that I may be too timid to ask in person” (G1797)  
                        - “AI made me feel more comfortable to learn compared to the teachers” (G2774)  
                        - “I sometimes feel reluctant to go to the teacher with questions, meanwhile I don’t feel this reluctance when going to AI.” (G3409)  

                        ### Code: Expert  
                        **Definition:** Has a position of knowledge or expertise.  
                        **Representative quotes:**  
                        - “They’re also more knowledgeable in terms of the subject” (G72)  
                        - “Teacher learning is helpful in the way that it is in person and they are knowledgeable in their field and are fully aware of the requirements and what they want the student to achieve.” (G64)  

                        ---

                        # Theme: Value  

                        ## Sub-theme: Value  

                        ### Code: Importance 
                        **Definition:** Is important or valuable.  
                        **Representative quotes:**  
                        - “Teacher is the most important point of learning” (G22)  
                        - “It didn’t impact my learning in any significant way as I always weighed the feedback from my tutors and lecturers above an AI.” (G1161)  
                        - “Nothing really changed, I still value teacher feedback more and I’ve never directly implemented a GenAI suggestion/feedback.” (G1185)  


                        """
                        
CODING_STRATEGY = """
    When coding student responses:

    1. **Surface-Level Understanding**  
    - Do not over-interpret student responses.  
    - Work from the most direct, surface-level meaning of the text.  
    - Avoid reading beyond what is explicitly written.

    2. **Chunk-Based Coding**  
    - Divide each response into meaningful chunks (a sentence or a significant phrase).  
    - A chunk should only be coded once, even if it could theoretically fit multiple codes.

    3. **Focus on Comparison**  
    - Prioritize identifying any comparison between AI feedback and teacher feedback.  
    - Look for explicit or implicit comparative language such as *more, less, better, worse, easier, harder, similar*.  

    4. **Actor + Comparator + Characteristic Logic**  
    - For each usable chunk, identify three components:  
        **Actor** – Who or what the comment refers to (A = AI, T = Teacher, B = Both).  
        **Comparator** – The relative position (e.g., more, less, similar, better, worse).  
        **Characteristic** – The feedback property being described (e.g., clarity, accuracy, relevance, timeliness, emotional safety).  
    - A valid coded chunk must include all three components.  

    5. **Handling Unusable Responses**  
    - If a response is unintelligible or does not appear to answer the question, code it as **UNUSABLE**.  
    - If a chunk is missing **any one** of the three required components (Actor, Comparator, Characteristic), code it as **UNUSABLE**.  
    - Example of unusable: "It was better" – we don’t know *which actor* it refers to.  

"""

OUTPUT_FORMAT = """
{
    "codes": [
        {   
            "actor": "...",
            "comparator": "...",
            "theme": "...",
            "sub_theme": "...",
            "code": "...",
            "reasoning": "..."
        }
    ]
}
"""

SHOTS = """

Examples of a good output are the next ones:

Example 1:

Question: "Please describe any differences in how it impacted your learning (comparing GenAI and your teacher)?"

Student response: "AI tends to be more uplifting or supportive than teachers, so when I ask for its opinion on something I’ve written it’ll compliment whereas my teacher will have edits to my work. Both are useful and important."

GOOD OUTPUT:

{
    "codes": [
        {   
            "actor": "A",
            "comparator": "More",
            "theme": "Tone",
            "sub_theme": "Positivity",
            "code": "Positivity",
            "reasoning": "The phrases 'more uplifting or supportive' and 'it’ll compliment' directly indicate positive affective framing. By definition, the Positivity code captures feedback that is polite, supportive, or 'people-pleasing', which matches the student’s description of AI’s tone relative to teachers."
        },
        {   
            "actor": "T",
            "comparator": "More",
            "theme": "Information",
            "sub_theme": "Quality",
            "code": "Specificity",
            "reasoning": "The contrast 'AI will compliment whereas my teacher will have edits' signals that teacher feedback contains actionable, text-level changes. This aligns with the Specificity code (detailed, targeted information) rather than general praise, indicating teachers provide more specific guidance."
        },
        {
            "actor": "B",
            "comparator": "Similar",
            "theme": "Value",
            "sub_theme": "Importance",
            "code": "Importance",
            "reasoning": "The closing statement 'Both are useful and important' is an explicit equivalence claim about value. The Importance code captures perceived significance; here the student assigns comparable importance to both AI and teacher feedback."
        }
    ]
}

Example 2:

Question: "Please describe any differences in how it impacted your learning (comparing GenAI and your teacher)?"

Student response: "Provides more immediate feedback while working on assignments when the teacher wasn't yet able to."

GOOD OUTPUT:

{
    "codes": [
        {   
            "actor": "A",
            "comparator": "More",
            "theme": "Processes",
            "sub_theme": "Access",
            "code": "Speed",
            "reasoning": "The term 'more immediate' directly maps to the Speed code (immediate/instant/time-efficient). The temporal clause 'while working on assignments' highlights availability during production (pre‑submission), reinforcing that AI’s response time outpaces the teacher’s in this context."
        }
    ]
}

Example 3:

Question: "Were there any differences in how the feedback made you feel (comparing GenAI and your teacher)?"

Student response: "AI made me feel good and teachers make me feel bad and dumb."

GOOD OUTPUT:

{
    "codes": [
        {   
            "actor": "A",
            "comparator": "More",
            "theme": "Feeling",
            "sub_theme": "Positive",
            "code": "Positive",
            "reasoning": "The literal statement 'AI made me feel good' is a direct indicator of positive emotional impact. The Positive code captures encouraging or motivating feelings, which the student explicitly attributes to AI in contrast to teachers."
        },
        {   
            "actor": "T",
            "comparator": "More",
            "theme": "Feeling",
            "sub_theme": "Negative",
            "code": "Negative",
            "reasoning": "The student reports teachers 'make me feel bad and dumb', which is an explicit negative affective response. The Negative code covers discouraging or frustrating emotions, indicating a stronger negative impact from teacher feedback relative to AI."
        }
    ]
}

"""

reasons_codes = """

                # Theme: Processes

                ## Sub-theme: Access

                ### Code: Unaware
                **Definition:** Respondents didn’t know GenAI could provide feedback; or didn’t know how to use it; or didn’t think of using it.  
                **Representative quotes:**
                - “Didn’t think to” (BC26)  
                - “I did not know that it could do that.” (BC58)  
                - “It has never occurred to me but I think it's a great idea” (BC8352)  
                - “Its hard to use.” (BC1077)  
                - “I don’t know how to do” (BC5731)  
                - “not sure how to use it to get feedback” (BC6041)

                ## Sub-theme: Effort

                ### Code: Effortful
                **Definition:** Too much effort to use AI or learn how to use AI; don’t have time to use AI (e.g., last-minute assignments).  
                **Representative quotes:**
                - “It is more effort than it's worth to program it to understand the confines and requirements of the work/project.” (BC7731)  
                - “I'm usually in a rush to submit work and don't have time.” (BC3034)

                ### Code: Less effort
                **Definition:** Off-loads effort or experience of learning, which in this context is not desirable.  
                **Representative quotes:**
                - “I'm paying substantial fees to undertake this course, I figure if I can't push myself to do the entirety of the work myself and mam relying on AI, then I am not gaining the skills I need to enter the workforce.” (BC2044)  
                - “Because I need to learn new things with some effort. Using GenAI for work is something like I'm not using my calibre to do the work. So in order to be part of this competitive world I should hold and thrive myself with my effort.” (BC2342)  
                - “It is because I want to use my own intelligence to learn something new instead of depending on AI and only get marks without knowing why.” (BC2857)

                ---

                # Theme: Information

                ## Sub-theme: Quality

                ### Code: Specificity
                **Definition:** Provides specific or detailed information. Antonyms: vague, generic.  
                **Representative quote:**
                - “I prefer specific feedback from what I have done, not general feedback that AI generated.” (BC1541)

                ### Code: In-depth
                **Definition:** Not enough, or sometimes not useful because it offers too much depth or additional explanation than desired.  
                **Representative quote:**
                - “I personally believe that GenAI lacks the nuance that would allow for effective feedback on a task.” (BC307)

                ### Code: Contextualised
                **Definition:** Information provided lacks the context of an assignment, rubrics, discipline or class.  
                **Representative quotes:**
                - “It does not know the specifics of the task I am doing” (BC62)  
                - “GenAI will not be the ones grading me. Furthermore, GenAI likely doesn't know the rubric for grading” (BC10)

                ### Code: Utility
                **Definition:** Useful, usable, or helpful information.  
                **Representative quotes:**
                - “I tried, but didn't find it very helpful” (BC5025)  
                - “I find it is not helpful in most circumstances as it tends to pull information from outside what I enter into the search engine.” (BC5060)  
                - “I feel like it's not going to help me in the feedback I need. It will just pick up on expression mistakes, which I can use through someone proof reading my work.” (BC7775)

                ### Code: Trustworthy
                **Definition:** Respondent does not trust the information; it is not reliable, or the provider/source is not competent to provide trustworthy information.  
                **Representative quotes:**
                - “I didn’t use GenAI for feedback on my work because I do not trust that it would be accurate” (BC3)  
                - “I would not trust GenAI to give me the correct answer or feedback” (BC17)  
                - “I do not trust the americanisation of Gen AI datasets” (BC807)  
                - “It's a machine, it doesn't know how to give feedback” (BC42)

                ## Sub-theme: Tone

                ### Code: Positivity
                **Definition:** Makes positive statements; people pleaser.  
                **Representative quote:**
                - “GenAI also tends to be a ‘yes man’ or otherwise reflect the opinion of the person using it” (BC203)

                ---

                # Theme: Relational

                ## Sub-theme: Relational

                ### Code: Personal
                **Definition:** Respondent values relationship or closer and embodied feedback; values the role or connection with humans.  
                **Representative quotes:**
                - “GenAI [can't] have a person-to-person conversation with me.” (BC10)  
                - “feedback should comes from professors” (BC2126)  
                - “For feedback I prefer to have a human give feedback” (BC2281)  
                - “GenAI lacks personal emotion, I believe in its professionalism but not its humanity.” (BC2422)

                ## Sub-theme: Relational

                ### Code: Expert
                **Definition:** Has a position of knowledge or expertise.  
                **Representative quotes:**
                - “I rather take feedback from a professional in the field” (BC5436)  
                - “Because GenAI does not hold the same human knowledge as a tut / lecturer holds, therefore does not provide the same insightful feedback.” (BC6441)

                ---

                # Theme: Value

                ## Sub-theme: Value

                ### Code: Preference
                **Definition:** Respondent preferred not using GenAI without specifying further reasons or expressing dislike.  
                **Representative quotes:**
                - “I've realized it writes in a way that I do not like.” (BC70)  
                - “I just didn't, I don't really like using GenAI” (BC35)

                ### Code: Need
                **Definition:** Does not want or need feedback; does not need feedback from AI (own skills or others are sufficient); no necessity to use AI for feedback.  
                **Representative quotes:**
                - “It was not necessary to use, all the tools that I needed were there” (BC1294)  
                - “I usually have ample opportunities for supervisors or tutors to provide feedback on my work, so I usually do not feel the need to request feedback from GenAI” (BC7872)

                ### Code: Unsustainable
                **Definition:** Respondent finds GenAI is not environmentally sustainable.  
                **Representative quotes:**
                - “I oppose the use of GenAI on environmental grounds” (BC201)  
                - “I personally avoid using GenAI as much as I can as its association with unethical practices and high environmental cost does not sit right with me.” (BC8539)

                ### Code: Privacy
                **Definition:** Respondent reports concern regarding privacy.  
                **Representative quotes:**
                - “I do not want my work added into GenAI datasets” (BC3)  
                - “I don't trust that the program won't store my work and later recycle it elsewhere.” (BC8563)  
                - “As I am in research, I do not want to break any confidentiality rules so I don't upload my work to third party sites for feedback.” (BC293)

                ### Code: Integrity
                **Definition:** Respondent committed to the originality of own work and ethical behaviour in academic work.  
                **Representative quotes:**
                - “I want to avoid having it influence my assignments.” (BC66)  
                - “I don't want to compromise the academic integrity of original ideas.” (BC71)  
                - “Not sure if it is cheating” (BC2137)  
                - “I'm afraid of breaking the rules.” (BC2161)  
                - “I always assumed the use of Gen AI was associated with risks of facing a rejection of my work so I have tried to steer clear from using it at all.” (BC2175)  
                - “it's not allowed” (BC2299)


                """
                
OUTPUT_FORMAT_43 =  """

{
    "reasons": [
        {   
            "theme": "...",
            "sub_theme": "...",
            "code": "...",
            "reasoning": "..."
        }
    ]
}                        
                    
                    
                    """
                    
SHOTS_43 = """

Examples of good output:

Example 1:

Question: "Why didn’t you use GenAI for feedback on your work?"

Student response: "I didn’t even know that this could be done. I also prefer to use my brain more to have creativity and originality in my work, studies and personal life."

Good output:

{
    "reasons": [
        {   
            "theme": "Processes",
            "sub_theme": "Access",
            "code": "Unaware",
            "reasoning": "The clause 'I didn’t even know that this could be done' is a direct admission of not knowing GenAI can provide feedback. The Unaware code explicitly covers respondents who did not know GenAI could do this or how to use it, so the statement maps precisely to Processes → Access → Unaware."
        },
        {
            "theme": "Value",
            "sub_theme": "Value",
            "code": "Preference",
            "reasoning": "The student states a normative stance—'I prefer to use my brain… for creativity and originality'—which indicates an intentional choice to avoid GenAI irrespective of its capabilities. This aligns with the Preference code, which captures a stated preference not to use GenAI without invoking a technical limitation."
        }
    ]
}

Example 2:

Question: "Why didn’t you use GenAI for feedback on your work?"
Student response: "I don't believe GenAI can provide more valuable feedback than I could get from a person proofreading it. I would rather ask a friend or family member to read what I've written."
Good output:
{
    "reasons": [
        {   
            "theme": "Information",
            "sub_theme": "Quality",
            "code": "Trustworthy",
            "reasoning": "By asserting 'I don't believe GenAI can provide more valuable feedback' and preferring human proofreaders, the student signals skepticism about GenAI’s credibility and evaluative competence. The Trustworthy code captures doubts about accuracy/reliability of information or the source’s ability to provide sound feedback, which is exactly what this statement conveys."
        }
    ]
}
"""

CODING_STRATEGY_43 = """

                        1) Scope & Principle

                        Goal: Identify explicit reasons for not using GenAI.

                        Surface-level only: Code the direct, literal meaning; do not infer beyond what is written.

                        2) Chunking

                        Split the response into meaningful chunks (sentence or key clause).

                        Assign one code per chunk. If multiple codes could apply, choose the most specific.

                        3) Required Fields

                        For each usable chunk, produce one reason with:

                        theme: Processes | Information | Tone | Relational | Value

                        sub_theme: Use the correct sub-theme; if not stated, repeat the theme (i.e., sub_theme = theme).

                        code: The specific characteristic (e.g., Unaware, Effortful, Less effort, Contextualised, Trustworthy, Privacy, Integrity, Unsustainable, Personal, Expert, Preference, Need, Positivity).

                        reasoning: Brief justification that quotes or precisely references the triggering phrase(s) and ties them to the code definition.
                        
                        4) Usability Rules

                        Usable: The chunk clearly states a reason for non-use (e.g., didn’t know, too hard/time, don’t trust, lacks context, not helpful, prefer human, no need, privacy, integrity, environmental concerns, style dislike).

                        UNUSABLE: Unintelligible, off-topic, or no reason stated.

                    """